## Evaluation 4D dynunet pipeline with NeuroI ROI dataset
* `train.py`with mode to `val` 을 기반으로함
* `commands/val.sh` 을 참고
* Brain image + mask image -> 4D modalities
* in_channels: 4, out_channels:1

In [2]:
import os
import numpy as np
import time

import torch
import torch.distributed as dist
from torch.nn.parallel import DistributedDataParallel
from monai.utils import first
from monai.handlers import (
    MeanDice,
    from_engine,
)
from monai.inferers import SimpleInferer, SlidingWindowInferer
# from monai.utils import set_determinism

from create_network import get_network_ke
from evaluator import (
    DynUNetEvaluator,
    DynUNetEvaluator_gpu,
    DynUNetEvaluator_SaveResult,
    DynUNetEvaluator_GPU_SaveResult_PostMapping
)
from config import get_config
from dataset_roi_4d import get_val_loader   # 4D modality test

In [4]:
config = "/data/kehyeong/project/MONAI_examples/dynunet_pipeline/config_roi_earlystop_toy_220209.yaml"
checkpoint = "/data/train/running/l/model_roi_try1_220217/models/net_key_metric=0.3331.pt"
val_dataset = "/work/NeuroI-models/ke-monai/data/roi/dataset_test_roi_toy2.csv"
val_output_dir = "./runs_eval2"

# tta_val = True    # ?!?!?!  whether to use test time augmentation.
tta_val = False

multi_gpu_flag = False
spacing = [1.0, 1.0, 1.0]
deep_supr_num = 3
window_mode = "gaussian"     # the mode parameter for SlidingWindowInferer.
eval_overlap = 0.5
amp = False
local_rank = 0

config = get_config(config)
data_dir = config["data_dir"]
image_file_path = config["image_file_path"]
label_file_path = config["label_file_path"]
mask_file_path = config["mask_file_path"]
val_batch_size = config["val"]["batch_size"] 
val_num_workers = config["val"]["num_workers"]
num_classes = config["num_classes"]
patch_size = config["patch_size"]

TTA는 말 그대로 Inference(Test) 과정에서 Augmentation 을 적용한 뒤 예측의 확률을 평균(또는 다른 방법)을 통해 도출하는 기법입니다. 모델 학습간에 다양한 Augmentation 을 적용하여 학습하였을시, Inference 과정에서도 유사한 Augmentation 을 적용하여 정확도를 높일 수 있습니다. 또는 이미지에 객체가 너무 작게 위치한 경우 원본 이미지, Crop 이미지를 넣는 등 다양하게 활용이 가능

In [5]:
if not os.path.exists(val_output_dir):
    os.makedirs(val_output_dir)
    
if multi_gpu_flag:
    dist.init_process_group(backend="nccl", init_method="env://")
    device = torch.device(f"cuda:{local_rank}")
    torch.cuda.set_device(device)
else:
    device = torch.device("cuda:6")
#     device = torch.device("cpu")

val_loader = get_val_loader(
    data_dir=data_dir,
    id_file=val_dataset,
    image_file_pattern=image_file_path,
    label_file_pattern=label_file_path,
    mask_file_pattern=mask_file_path,
    batch_size=val_batch_size,
    num_workers=val_num_workers,
    multi_gpu_flag=multi_gpu_flag
)

{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0303 00_0
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0303 01_d
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0197 00_0
{dir}/s_{id}_{aid}_b.nii.gz /data/train/running/l/input_augmented SU0197 01_d


Loading dataset: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:23<00:00,  5.91s/it]


In [6]:
test_data = first(val_loader)
print(test_data.keys())
print("image shape:", test_data['image'].shape)
print("image dtype:", test_data['image'].dtype)
print("label shape:", test_data['label'].shape)
print("label dtype:", test_data['label'].dtype)
print("1번 배치의 유니크한 라벨 리스트:", np.unique(test_data['label']))
total_labels = np.unique(test_data['label'])
print(f'1번 배치의 유니크한 라벨 class 수: {len(total_labels)}')

dict_keys(['image', 'label', 'mask', 'image_meta_dict', 'label_meta_dict', 'mask_meta_dict', 'image_transforms', 'label_transforms', 'mask_transforms', 'original_shape', 'bbox', 'crop_shape', 'resample_flag', 'anisotrophy_flag'])
image shape: torch.Size([1, 2, 154, 154, 152])
image dtype: torch.float32
label shape: torch.Size([1, 1, 186, 230, 230])
label dtype: torch.uint8
1번 배치의 유니크한 라벨 리스트: [  0   1   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  20  21  23  24  25  26  27  28  29  30  31  32  33  34  36  37  39  40
  41  42  43  44  45  46  47  48  49  50  51  52  53  54  55  56  57  58
  59  60  61  62  63  64  65  66  67  68  69  70  71  72  73  74  75  76
  77  78  79  80  81  82  83  84  85  86  87  88  89  90  91  92  93  94
  95  96  97  98  99 100 101 102 103 104 105 106 107 108]
1번 배치의 유니크한 라벨 class 수: 104


In [7]:
spacings = test_data['image_meta_dict']["pixdim"]
spacings

tensor([[1., 1., 1., 1., 0., 0., 0., 0.]])

In [9]:
test_data['image_meta_dict']["pixdim"][0][1:4]

tensor([1., 1., 1.])

In [20]:
filename_list = []
for each in val_loader:
    filename = each['image_meta_dict']['filename_or_obj'][0].split("/")[-1].split(".")[0]
    filename_list.append(filename)
filename_list

['s_SU0303_00_0_b', 's_SU0303_01_d_b', 's_SU0197_00_0_b', 's_SU0197_01_d_b']

In [ ]:
test_data['image_meta_dict']['filename_or_obj']

In [6]:
filename = test_data['image_meta_dict']['filename_or_obj'][0].split("/")[-1]
filename

's_SU0303_00_0_b.nii.gz'

--------------

post transform 테스트

In [ ]:
from monai.transforms import AsDiscrete, Transform
post_label = AsDiscrete(to_onehot=num_classes)
post_pred = AsDiscrete(argmax=True, to_onehot=num_classes)

In [ ]:
aaa = post_label(test_data['label'][0])
aaa.shape

In [ ]:
aaa

In [ ]:
aaa[:, 0, 0, 0]

In [ ]:
test_data['label'].shape

In [ ]:
test_data['label'][0, :, 80, 80, 80]

In [ ]:
aaa[:, 80, 80, 80]

In [ ]:
bbb = post_pred(aaa)
bbb.shape

In [ ]:
bbb[:, 80, 80, 80]

------------------------------

In [7]:
properties = {
    'modality': [0,1],
    'labels': np.arange(num_classes)
}
n_class = len(properties["labels"])
in_channels = len(properties["modality"])
in_channels, n_class

(2, 109)

In [8]:
# produce the network
net = get_network_ke(properties, patch_size, spacing, deep_supr_num, 
                     val_output_dir, checkpoint)    # val_output_dir은 삭제 필요. 안씀
print('Loading nnUNET Done!!!')
net = net.to(device)
print('Loading nnUNET to GPU devices Done!!!')
print(net)

pretrained checkpoint: /data/train/running/l/model_roi_try1_220217/models/net_key_metric=0.3331.pt loaded
Loading nnUNET Done!!!
Loading nnUNET to GPU devices Done!!!
DynUNet(
  (input_block): UnetBasicBlock(
    (conv1): Convolution(
      (conv): Conv3d(2, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    )
    (conv2): Convolution(
      (conv): Conv3d(32, 32, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1), bias=False)
    )
    (lrelu): LeakyReLU(negative_slope=0.01, inplace=True)
    (norm1): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
    (norm2): InstanceNorm3d(32, eps=1e-05, momentum=0.1, affine=False, track_running_stats=False)
  )
  (downsamples): ModuleList(
    (0): UnetBasicBlock(
      (conv1): Convolution(
        (conv): Conv3d(32, 64, kernel_size=(3, 3, 3), stride=(2, 2, 2), padding=(1, 1, 1), bias=False)
      )
      (conv2): Convolution(
        (conv): Conv3d(64, 64, kernel_size=(3, 

In [9]:
if multi_gpu_flag:
    net = DistributedDataParallel(module=net, device_ids=[device])

num_classes = len(properties["labels"])

In [10]:
device

device(type='cuda', index=6)

In [ ]:
# ## 1. 기본형
# net.eval()
# evaluator = DynUNetEvaluator(
#     device=device,
#     val_data_loader=val_loader,
#     network=net,
#     num_classes=num_classes,
#     inferer=SlidingWindowInferer(
#         roi_size=patch_size,
#         sw_batch_size=val_batch_size,
#         overlap=eval_overlap,
#         mode=window_mode,
#     ),
#     postprocessing=None,      # 이걸 바꿔줘야할듯...
#     key_val_metric={
#         "val_mean_dice": MeanDice(
#             include_background=False,
#             output_transform=from_engine(["pred", "label"]),
#         )
#     },
#     additional_metrics=None,
#     amp=amp,
#     tta_val=tta_val,
#     orig_label_classes=orig_label_classes,
#     target_label_classes=target_label_classes
# )

In [ ]:
# ## 2. GPU inference 형
# net.eval()
# evaluator = DynUNetEvaluator_gpu(
#     device=device,
#     val_data_loader=val_loader,
#     network=net,
#     num_classes=num_classes,
#     inferer=SlidingWindowInferer(
#         roi_size=patch_size,
#         sw_batch_size=val_batch_size,
#         overlap=eval_overlap,
#         mode=window_mode,
#     ),
#     postprocessing=None,      # 이걸 바꿔줘야할듯...
#     key_val_metric={
#         "val_mean_dice": MeanDice(
#             include_background=False,
#             output_transform=from_engine(["pred", "label"]),
#         )
#     },
#     additional_metrics=None,
#     amp=amp,
#     tta_val=tta_val,
#     orig_label_classes=orig_label_classes,
#     target_label_classes=target_label_classes
# )

In [ ]:
# ## 3. GPU ver. + inference image 저장 + tta False

# net.eval()
# evaluator = DynUNetEvaluator_SaveResult(
#     device=device,
#     val_data_loader=val_loader,
#     network=net,
#     output_dir=val_output_dir,
#     num_classes=num_classes,
#     inferer=SlidingWindowInferer(
#         roi_size=patch_size,
#         sw_batch_size=val_batch_size,
#         overlap=eval_overlap,
#         mode=window_mode,
#     ),
#     postprocessing=None,      # 이걸 바꿔줘야할듯...
#     key_val_metric={
#         "val_mean_dice": MeanDice(
#             include_background=False,
#             output_transform=from_engine(["pred", "label"]),
#         )
#     },
#     additional_metrics=None,
#     amp=amp,
#     tta_val=tta_val,
#     orig_label_classes=orig_label_classes,
#     target_label_classes=target_label_classes
# )

In [ ]:
# ## 4. GPU ver. + inference image 저장형 + tta False + posttransform (mapping index)
# from monai.transforms import (
#     LoadImaged,
#     AddChanneld,
#     MapLabelValued,
#     Compose
# )

# orig_label_classes, target_label_classes = (
#     np.array([   0,    2,    3,    4,    5,    7,    8,   10,   11,   12,   13,
#          14,   15,   16,   17,   18,   24,   26,   28,   30,   31,   41,
#          42,   43,   44,   46,   47,   49,   50,   51,   52,   53,   54,
#          58,   60,   62,   63,   77,   80,   85,  251,  252,  253,  254,
#         255, 1000, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011,
#        1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
#        1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, 1035,
#        2000, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
#        2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
#        2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2034, 2035]),
#     np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
#         13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
#         26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
#         39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
#         52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
#         65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
#         78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
#         91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
#        104, 105, 106, 107, 108])
# )

# post_trans = MapLabelValued(
#     keys=["pred"], 
#     orig_labels=target_label_classes, 
#     target_labels=orig_label_classes
# )


# net.eval()
# evaluator = DynUNetEvaluator_SaveResult(
#     device=device,
#     val_data_loader=val_loader,
#     network=net,
#     output_dir=val_output_dir,
#     num_classes=num_classes,
#     inferer=SlidingWindowInferer(
#         roi_size=patch_size,
#         sw_batch_size=val_batch_size,
#         overlap=eval_overlap,
#         mode=window_mode,
#     ),
#     postprocessing=post_trans,      # 이걸 바꿔줘야할듯...
#     key_val_metric={
#         "val_mean_dice": MeanDice(
#             include_background=False,
#             output_transform=from_engine(["pred", "label"]),
#         )
#     },
#     additional_metrics=None,
#     amp=amp,
#     tta_val=tta_val,
# )

In [11]:
## 5. GPU ver. + inference image 저장형 + tta False + posttransform 2 (mapping index)
# evaluator class 내부에서 transform

orig_label_classes, target_label_classes = (
    np.array([   0,    2,    3,    4,    5,    7,    8,   10,   11,   12,   13,
         14,   15,   16,   17,   18,   24,   26,   28,   30,   31,   41,
         42,   43,   44,   46,   47,   49,   50,   51,   52,   53,   54,
         58,   60,   62,   63,   77,   80,   85,  251,  252,  253,  254,
        255, 1000, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011,
       1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
       1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, 1035,
       2000, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
       2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2034, 2035]),
    np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108])
)

net.eval()
evaluator = DynUNetEvaluator_GPU_SaveResult_PostMapping(
    device=device,
    val_data_loader=val_loader,
    network=net,
    output_dir=val_output_dir,
    num_classes=num_classes,
    inferer=SlidingWindowInferer(
        roi_size=patch_size,
        sw_batch_size=val_batch_size,
        overlap=eval_overlap,
        mode=window_mode,
    ),
    postprocessing=None,
    key_val_metric={
        "val_mean_dice": MeanDice(
            include_background=False,
            output_transform=from_engine(["pred", "label"]),
        )
    },
    additional_metrics=None,
    amp=amp,
    tta_val=tta_val,
    orig_label_classes=orig_label_classes,
    target_label_classes=target_label_classes
)

In [12]:
start = time.time()
evaluator.run()
print("time :", time.time() - start)

<class 'torch.Tensor'>
torch.Size([109, 154, 154, 152])
torch.Size([154, 154, 152])
save s_SU0303_00_0_b.nii.gz with shape: (186, 230, 230)
MRI img:s_SU0303_00_0_b.nii.gz eval done .................
<class 'torch.Tensor'>
torch.Size([109, 145, 156, 159])
torch.Size([145, 156, 159])
save s_SU0303_01_d_b.nii.gz with shape: (186, 230, 230)
MRI img:s_SU0303_01_d_b.nii.gz eval done .................
<class 'torch.Tensor'>
torch.Size([109, 141, 146, 133])
torch.Size([141, 146, 133])
save s_SU0197_00_0_b.nii.gz with shape: (186, 230, 230)
MRI img:s_SU0197_00_0_b.nii.gz eval done .................
<class 'torch.Tensor'>
torch.Size([109, 154, 153, 121])
torch.Size([154, 153, 121])
save s_SU0197_01_d_b.nii.gz with shape: (186, 230, 230)
MRI img:s_SU0197_01_d_b.nii.gz eval done .................
time : 471.54902839660645


----

이미지를 로드하여 mapping transform 테스트를 진행해야 한다.

In [ ]:
from monai.transforms import (
    LoadImaged,
    AddChanneld,
    MapLabelValued,
    Compose
)
from monai.data import (
    CacheDataset,
    DataLoader,
    partition_dataset,
)

orig_label_classes, target_label_classes = (
    np.array([   0,    2,    3,    4,    5,    7,    8,   10,   11,   12,   13,
         14,   15,   16,   17,   18,   24,   26,   28,   30,   31,   41,
         42,   43,   44,   46,   47,   49,   50,   51,   52,   53,   54,
         58,   60,   62,   63,   77,   80,   85,  251,  252,  253,  254,
        255, 1000, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011,
       1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
       1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, 1035,
       2000, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
       2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2034, 2035]),
    np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108])
)

def get_test_transform():
    keys = ["label", "pred"]
    transforms = [
        LoadImaged(keys=keys),
#         AddChanneld(keys=keys),
#         MapLabelValued(
#             keys=["pred"], 
#             orig_labels=target_label_classes, 
#             target_labels=orig_label_classes
#         ),
#         Orientationd(keys=keys, axcodes="RAS"),
#         ConcatItemsd(keys=["image", "mask"], name="image"),
#         PreprocessAnisotropic(
#             keys=["image"],
#             clip_values=clip_values,
#             pixdim=spacing,
#             normalize_values=normalize_values,
#             model_mode="test",
#         ),
#         CastToTyped(keys=["image"], dtype=(np.float32)),
#         EnsureTyped(keys=["image"]),
    ]
    return Compose(transforms)
transform = get_test_transform()

In [ ]:
# import shutil
# original_file = os.path.join(data_dir, "s_SU0197_00_0_b.nii.gz")
# shutil.copy(original_file, './')

In [ ]:
data = []
label_file = os.path.join(data_dir, "s_SU0197_00_0_l.nii.gz")
pred_file = "runs_eval2/s_SU0197_00_0_b.nii.gz"
data.append({'label': label_file, 'pred': pred_file})

dataset = CacheDataset(
    data=data,
    transform=transform,
    num_workers=8,
    cache_rate=1.0,
)

data_loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    num_workers=4,
    drop_last=True,
)

In [ ]:
test_data = first(data_loader)

In [ ]:
test_data.keys()

In [ ]:
affine = test_data["pred_meta_dict"]["affine"].numpy()[0]
affine

In [ ]:
test_data['label'].shape, test_data['pred'].shape

In [ ]:
test_data['pred'].dtype, test_data['label'].dtype

In [ ]:
len(np.unique(test_data['pred'])), np.unique(test_data['pred'])

In [ ]:
np.unique(test_data['label'])

In [ ]:
result = np.where(test_data['pred'] == 2018)   # 2018, 93
result

In [ ]:
idx = 12
test_data['pred'][result[0][idx], result[1][idx], result[2][idx], result[3][idx]]

In [ ]:
test_data['label'][result[0][idx], result[1][idx], result[2][idx], result[3][idx]]

오케 맵핑이 잘되고 있음을 확인하였음.

------------------
저장된 이미지로부터 post-transform mapping 하여 결과 확인해본다.

In [ ]:
from monai.transforms import MapLabelValue
orig_label_classes, target_label_classes = (
    np.array([   0,    2,    3,    4,    5,    7,    8,   10,   11,   12,   13,
        14,   15,   16,   17,   18,   24,   26,   28,   30,   31,   41,
        42,   43,   44,   46,   47,   49,   50,   51,   52,   53,   54,
        58,   60,   62,   63,   77,   80,   85,  251,  252,  253,  254,
        255, 1000, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011,
    1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
    1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, 1035,
    2000, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
    2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
    2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2034, 2035]),
    np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
    104, 105, 106, 107, 108])
)
post_trans = MapLabelValue(    # 0~108로 잡혀있는 label을 original index로 변경
    orig_labels=target_label_classes, 
    target_labels=orig_label_classes
)

In [ ]:
test_data['pred'].shape, test_data['label'].shape

In [ ]:
aaa = post_trans(test_data['pred'])
aaa.shape

In [ ]:
# aaa = test_data['pred']

In [ ]:
result = np.where(aaa == 2018)   # 41
result

In [ ]:
idx = 226
aaa[result[0][idx], result[1][idx], result[2][idx], result[3][idx]]

In [ ]:
test_data['label'][result[0][idx], result[1][idx], result[2][idx], result[3][idx]]

post transform 한 결과를 이미지로 저장한다.

In [ ]:
aaa.shape

In [ ]:
aaa.squeeze().shape

In [ ]:
from monai.data.nifti_writer import write_nifti

In [ ]:
write_nifti(
        data=aaa.squeeze(),
        file_name='aa2.nii.gz',
        affine=affine,
        resample=False,
        output_dtype=np.uint8,
    )

In [ ]:
aaa.shape

In [ ]:
from monai.transforms import (
    LoadImaged,
    AddChanneld,
    MapLabelValued,
    Compose
)
from monai.data import (
    CacheDataset,
    DataLoader,
    partition_dataset,
)

orig_label_classes, target_label_classes = (
    np.array([   0,    2,    3,    4,    5,    7,    8,   10,   11,   12,   13,
         14,   15,   16,   17,   18,   24,   26,   28,   30,   31,   41,
         42,   43,   44,   46,   47,   49,   50,   51,   52,   53,   54,
         58,   60,   62,   63,   77,   80,   85,  251,  252,  253,  254,
        255, 1000, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011,
       1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
       1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, 1035,
       2000, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
       2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
       2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2034, 2035]),
    np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108])
)

def get_test_transform():
    keys = ["label", "pred"]
    transforms = [
        LoadImaged(keys=keys),
#         AddChanneld(keys=keys),
#         MapLabelValued(
#             keys=["pred"], 
#             orig_labels=target_label_classes, 
#             target_labels=orig_label_classes
#         ),
#         Orientationd(keys=keys, axcodes="RAS"),
#         ConcatItemsd(keys=["image", "mask"], name="image"),
#         PreprocessAnisotropic(
#             keys=["image"],
#             clip_values=clip_values,
#             pixdim=spacing,
#             normalize_values=normalize_values,
#             model_mode="test",
#         ),
#         CastToTyped(keys=["image"], dtype=(np.float32)),
#         EnsureTyped(keys=["image"]),
    ]
    return Compose(transforms)
transform = get_test_transform()

In [ ]:
data = []
label_file = os.path.join(data_dir, "s_SU0197_00_0_l.nii.gz")
pred_file = "aa2.nii.gz"
data.append({'label': label_file, 'pred': pred_file})

dataset = CacheDataset(
    data=data,
    transform=transform,
    num_workers=8,
    cache_rate=1.0,
)

data_loader = DataLoader(
    dataset,
    batch_size=1,
    shuffle=True,
    num_workers=4,
    drop_last=True,
)

In [ ]:
test_data = first(data_loader)

In [ ]:
test_data.keys()

In [ ]:
result = np.where(test_data['pred'] == 2018)
result

In [ ]:
idx = 12
test_data['pred'][result[0][idx], result[1][idx], result[2][idx], result[3][idx]]

In [ ]:
test_data['label'][result[0][idx], result[1][idx], result[2][idx], result[3][idx]]

## finding 정리
evaluator에 MeanDice가 저장되는 개념.
 * evaluator.state.metrics : 전체 ROI MeanDice 평균
 * evaluator.state.metric_details["val_mean_dice"] : ROI별 MeanDice 평균

In [ ]:
if local_rank == 0:
    print(evaluator.state.metrics)
    results = evaluator.state.metric_details["val_mean_dice"]
    if num_classes > 2:
        for i in range(num_classes - 1):
            print(
                "mean dice for label {} is {}".format(i + 1, results[:, i].mean())
            )

In [ ]:
dir(evaluator)

In [ ]:
evaluator.state_dict()

In [ ]:
attr_list = dir(evaluator.state)
for each in attr_list:
    print(each)
    each = eval(each)
    print(evaluator.each)

In [ ]:
evaluator.orig_label_classes

In [ ]:
attr_list = evaluator.__dir__()
attr_list

In [ ]:
i = 2
print(attr_list[i])
evaluator.state

In [ ]:
evaluator.state.output[0].keys()

In [ ]:
# meandice 계산 하려면 이렇게 해야함.
evaluator.state.output[0]['label'].shape, evaluator.state.output[0]['pred'].shape

In [ ]:
# meandice 계산 하려면 이렇게 해야함.
evaluator.state.output[0]['label'].shape, evaluator.state.output[0]['pred'].shape

In [ ]:
for a in evaluator.state.dataloader:
    print(a)

In [ ]:
dir(evaluator.state)

In [ ]:
type(evaluator.state.metric_details), evaluator.state.metric_details.keys()

In [ ]:
evaluator.state.metric_details['val_mean_dice']

In [ ]:
evaluator.state.metric_details['val_mean_dice'].shape

----
### metric - MeanDice 정리 code 작성

In [ ]:
import pandas as pd

In [ ]:
results = evaluator.state.metric_details["val_mean_dice"]
results = np.array(results)
results

In [ ]:
results.shape

In [ ]:
num_classes = results.shape[1]

df = pd.DataFrame(results).T
df['mean'] = df.mean(axis=1)
df

In [ ]:
# 그냥 평균을 내면 안대고 0인 값들은 계산이 안된거니까 빼놓고 평균을 구해야 한다.
test = df[df['mean'] > 0]
test

In [ ]:
test.shape

In [ ]:
np.zeros(test.shape[1])

In [ ]:
aa = pd.DataFrame(np.zeros(test.shape[1])).T
aa

In [ ]:
aa

In [ ]:
test

In [ ]:
pd.concat([pd.Series([0]), test])

In [ ]:
pd.concat([pd.DataFrame(np.zeros(test.shape[1])).T, test])

In [ ]:
test['mean'].mean()

In [ ]:
df['total_mean'] = evaluator.state.metrics['val_mean_dice']

In [ ]:
df

In [ ]:
orig_label_classes, target_label_classes = (
    np.array([   0,    2,    3,    4,    5,    7,    8,   10,   11,   12,   13,
        14,   15,   16,   17,   18,   24,   26,   28,   30,   31,   41,
        42,   43,   44,   46,   47,   49,   50,   51,   52,   53,   54,
        58,   60,   62,   63,   77,   80,   85,  251,  252,  253,  254,
        255, 1000, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011,
    1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
    1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, 1035,
    2000, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
    2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
    2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2034, 2035]),
    np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
    104, 105, 106, 107, 108])
)

In [ ]:
df_index = pd.DataFrame([orig_label_classes, target_label_classes]).T
df_index.columns = ['original_index', 'target_index']
df_index

In [ ]:
df_mid = pd.concat([df_index, df], axis=1)
df_mid

In [ ]:
mapping_f = './ROI_mapping_final_220314.txt'
df_name_map = pd.read_csv(mapping_f, sep='\t')
df_name_map

In [ ]:
df_final = pd.merge(df_name_map, df_mid, how='right', left_on="label_label-from", right_on="original_index")
df_final.drop(columns=["label_label-from", "label_label-to"], inplace=True)
df_final

## 한셀 코딩

In [21]:
filename_list = []
for each in val_loader:
    filename = each['image_meta_dict']['filename_or_obj'][0].split("/")[-1].split(".")[0]
    filename_list.append(filename)
filename_list

['s_SU0303_00_0_b', 's_SU0303_01_d_b', 's_SU0197_00_0_b', 's_SU0197_01_d_b']

In [24]:
import pandas as pd

orig_label_classes, target_label_classes = (
    np.array([   0,    2,    3,    4,    5,    7,    8,   10,   11,   12,   13,
        14,   15,   16,   17,   18,   24,   26,   28,   30,   31,   41,
        42,   43,   44,   46,   47,   49,   50,   51,   52,   53,   54,
        58,   60,   62,   63,   77,   80,   85,  251,  252,  253,  254,
        255, 1000, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011,
    1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
    1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, 1035,
    2000, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
    2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
    2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2034, 2035]),
    np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
    104, 105, 106, 107, 108])
)

# GPU to CPU
results = evaluator.state.metric_details["val_mean_dice"]
results = np.array(results)

num_classes = results.shape[1]

# add ROIs Means & total Mean
df = pd.DataFrame(results).T
df = pd.concat([pd.DataFrame(np.zeros(df.shape[1])).T, df])    # add background as 0
df.columns = filename_list
# df
df['roi_meanDice'] = df.mean(axis=1)
df['total_meanDice'] = evaluator.state.metrics['val_mean_dice']
df.reset_index(inplace=True, drop=True)

# add orig_index, target_index
df_index = pd.DataFrame([orig_label_classes, target_label_classes]).T
df_index.columns = ['original_index', 'target_index']
df_mid = pd.concat([df_index, df], axis=1)

# mapping original name
mapping_f = './ROI_mapping_final_220314.txt'
df_name_map = pd.read_csv(mapping_f, sep='\t')
df_final = pd.merge(df_name_map, df_mid, how='right', left_on="label_label-from", right_on="original_index")
df_final.drop(columns=["label_label-from", "label_label-to"], inplace=True)


df_final.to_csv('eval_result.txt', index=False)
df_final

,ori_name,original_index,target_index,s_SU0303_00_0_b,s_SU0303_01_d_b,s_SU0197_00_0_b,s_SU0197_01_d_b,roi_meanDice,total_meanDice
0,Clear Label,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.875803
1,L Cerebral-White-Matter,2,1,0.955718,0.940041,0.956269,0.934953,0.946745,0.875803
2,L Cerebral-Cortex,3,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.875803
3,L Lateral-Ventricle,4,3,0.961130,0.931103,0.969748,0.933617,0.948900,0.875803
4,L Inf-Lat-Vent,5,4,0.735562,0.634304,0.837267,0.741993,0.737282,0.875803
...,...,...,...,...,...,...,...,...,...
104,R superiorparietal,2029,104,0.919095,0.876818,0.906960,0.885491,0.897091,0.875803
105,R superiortemporal,2030,105,0.943973,0.922981,0.949100,0.922715,0.934692,0.875803
106,R supramarginal,2031,106,0.929319,0.904306,0.926189,0.896883,0.914174,0.875803
107,R transversetemporal,2034,107,0.879171,0.811189,0.845006,0.786370,0.830434,0.875803


In [14]:
import pandas as pd

orig_label_classes, target_label_classes = (
    np.array([   0,    2,    3,    4,    5,    7,    8,   10,   11,   12,   13,
        14,   15,   16,   17,   18,   24,   26,   28,   30,   31,   41,
        42,   43,   44,   46,   47,   49,   50,   51,   52,   53,   54,
        58,   60,   62,   63,   77,   80,   85,  251,  252,  253,  254,
        255, 1000, 1002, 1003, 1005, 1006, 1007, 1008, 1009, 1010, 1011,
    1012, 1013, 1014, 1015, 1016, 1017, 1018, 1019, 1020, 1021, 1022,
    1023, 1024, 1025, 1026, 1027, 1028, 1029, 1030, 1031, 1034, 1035,
    2000, 2002, 2003, 2005, 2006, 2007, 2008, 2009, 2010, 2011, 2012,
    2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022, 2023,
    2024, 2025, 2026, 2027, 2028, 2029, 2030, 2031, 2034, 2035]),
    np.array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
    104, 105, 106, 107, 108])
)

# GPU to CPU
results = evaluator.state.metric_details["val_mean_dice"]
results = np.array(results)

num_classes = results.shape[1]

# add ROIs Means & total Mean
df = pd.DataFrame(results).T
df = pd.concat([pd.DataFrame(np.zeros(df.shape[1])).T, df])    # add background as 0
df['roi_meanDice'] = df.mean(axis=1)
df['total_meanDice'] = evaluator.state.metrics['val_mean_dice']
df.reset_index(inplace=True, drop=True)

# add orig_index, target_index
df_index = pd.DataFrame([orig_label_classes, target_label_classes]).T
df_index.columns = ['original_index', 'target_index']
df_mid = pd.concat([df_index, df], axis=1)

# mapping original name
mapping_f = './ROI_mapping_final_220314.txt'
df_name_map = pd.read_csv(mapping_f, sep='\t')
df_final = pd.merge(df_name_map, df_mid, how='right', left_on="label_label-from", right_on="original_index")
df_final.drop(columns=["label_label-from", "label_label-to"], inplace=True)


df_final.to_csv('eval_result.txt', index=False)
df_final

,ori_name,original_index,target_index,0,1,2,3,roi_meanDice,total_meanDice
0,Clear Label,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.875803
1,L Cerebral-White-Matter,2,1,0.955718,0.940041,0.956269,0.934953,0.946745,0.875803
2,L Cerebral-Cortex,3,2,0.000000,0.000000,0.000000,0.000000,0.000000,0.875803
3,L Lateral-Ventricle,4,3,0.961130,0.931103,0.969748,0.933617,0.948900,0.875803
4,L Inf-Lat-Vent,5,4,0.735562,0.634304,0.837267,0.741993,0.737282,0.875803
...,...,...,...,...,...,...,...,...,...
104,R superiorparietal,2029,104,0.919095,0.876818,0.906960,0.885491,0.897091,0.875803
105,R superiortemporal,2030,105,0.943973,0.922981,0.949100,0.922715,0.934692,0.875803
106,R supramarginal,2031,106,0.929319,0.904306,0.926189,0.896883,0.914174,0.875803
107,R transversetemporal,2034,107,0.879171,0.811189,0.845006,0.786370,0.830434,0.875803


In [15]:
df_tf = df_final.T
df_tf

,0,1,2,3,4,5,6,7,8,9,...,99,100,101,102,103,104,105,106,107,108
ori_name,Clear Label,L Cerebral-White-Matter,L Cerebral-Cortex,L Lateral-Ventricle,L Inf-Lat-Vent,L Cerebellum-White-Matter,L Cerebellum-Cortex,L Thalamus-Proper,L Caudate,L Putamen,...,R precentral,R precuneus,R rostralanteriorcingulate,R rostralmiddlefrontal,R superiorfrontal,R superiorparietal,R superiortemporal,R supramarginal,R transversetemporal,R insula
original_index,0,2,3,4,5,7,8,10,11,12,...,2024,2025,2026,2027,2028,2029,2030,2031,2034,2035
target_index,0,1,2,3,4,5,6,7,8,9,...,99,100,101,102,103,104,105,106,107,108
0,0.0,0.955718,0.0,0.96113,0.735562,0.926234,0.972038,0.958088,0.925768,0.855526,...,0.894897,0.925008,0.898484,0.926971,0.94249,0.919095,0.943973,0.929319,0.879171,0.932533
1,0.0,0.940041,0.0,0.931103,0.634304,0.913915,0.966423,0.944151,0.876152,0.852786,...,0.871936,0.894403,0.881671,0.9007,0.921151,0.876818,0.922981,0.904306,0.811189,0.908294
2,0.0,0.956269,0.0,0.969748,0.837267,0.927647,0.975431,0.939587,0.935353,0.846236,...,0.915736,0.912142,0.920602,0.917094,0.949422,0.90696,0.9491,0.926189,0.845006,0.94661
3,0.0,0.934953,0.0,0.933617,0.741993,0.910055,0.971316,0.91186,0.890086,0.81645,...,0.905135,0.88495,0.839076,0.900118,0.934494,0.885491,0.922715,0.896883,0.78637,0.891442
roi_meanDice,0.0,0.946745,0.0,0.9489,0.737282,0.919463,0.971302,0.938421,0.90684,0.84275,...,0.896926,0.904126,0.884958,0.911221,0.936889,0.897091,0.934692,0.914174,0.830434,0.91972
total_meanDice,0.875803,0.875803,0.875803,0.875803,0.875803,0.875803,0.875803,0.875803,0.875803,0.875803,...,0.875803,0.875803,0.875803,0.875803,0.875803,0.875803,0.875803,0.875803,0.875803,0.875803


In [ ]:
# df_tf.reset_index(inplace=True)

In [ ]:
new_header = df_tf.iloc[0,:]
df_tf = df_tf[1:]
df_tf.columns = new_header

df_tf

In [ ]:
df_tf.to_csv('eval_result_transpose.csv')